In [1]:
import chess
import numpy as np
import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
import pandas as pd
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
path_csv = 'D:/AI/Chess/ChessApp/Chess/csv_data/morphy'
# Ví dụ nếu bạn tải lên Colab:
# path_csv = '/content/csv_data/morphy'

if not os.path.isdir(path_csv):
    print(f"Lỗi: Thư mục không tồn tại: {path_csv}")
    print("Vui lòng kiểm tra lại đường dẫn `path_csv`.")
    # Dừng thực thi nếu thư mục không đúng (trong môi trường notebook có thể không dừng hẳn)
    raise FileNotFoundError(f"Thư mục không tồn tại: {path_csv}")
else:
    files_csv = glob.glob(os.path.join(path_csv, "*.csv")) # Dùng os.path.join cho an toàn

    if not files_csv:
        print(f"Lỗi: Không tìm thấy file CSV nào trong: {path_csv}")
        raise FileNotFoundError(f"Không tìm thấy file CSV nào trong: {path_csv}")
    else:
        li = []
        print(f"Đang đọc {len(files_csv)} file CSV từ {path_csv}...")
        for filename in files_csv:
            try:
                df = pd.read_csv(filename, index_col=None, header=0)
                li.append(df)
            except Exception as e:
                print(f"Lỗi khi đọc file {filename}: {e}")

        if not li:
             print(f"Lỗi: Không đọc được dữ liệu từ bất kỳ file CSV nào.")
             raise ValueError("Không có dữ liệu để xử lý.")
        else:
            train_df = pd.concat(li, axis=0, ignore_index=True)
            print("Đọc dữ liệu hoàn tất.")



Đang đọc 193 file CSV từ D:/AI/Chess/ChessApp/Chess/csv_data/morphy...
Đọc dữ liệu hoàn tất.


In [3]:
train_df = shuffle(train_df, random_state=42)

In [4]:
board_columns = list(train_df.iloc[:, 0:64].columns)
label_column = 'good_move'

In [5]:
# Định nghĩa ánh xạ quân cờ sang kênh
piece_to_channel = {
    'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5, # White
    'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11 # Black
}
NUM_CHANNELS = 12
# Từ điển ánh xạ ký hiệu quân cờ PGN sang ký hiệu dùng trong dict
pgn_map = {
    'wP': 'P', 'wN': 'N', 'wB': 'B', 'wR': 'R', 'wQ': 'Q', 'wK': 'K',
    'bP': 'p', 'bN': 'n', 'bB': 'b', 'bR': 'r', 'bQ': 'q', 'bK': 'k',
    '--': None, 'EMPTY': None # Ô trống
}

# Tạo map từ tên cột (a1, h8) sang index (0,0) -> (7,7)
square_name_to_index = {name: (r, f) for r, rank_name in enumerate('12345678')
                       for f, file_name in enumerate('abcdefgh')
                       for name in [file_name + rank_name]}

In [6]:
def board_series_to_cnn_array(board_series):
    """Chuyển đổi một hàng (Series) 64 cột trạng thái bàn cờ thành (8, 8, 12) array."""
    cnn_array = np.zeros((8, 8, NUM_CHANNELS), dtype=np.float32)
    for square_name, pgn_piece in board_series.items():
        piece_symbol = pgn_map.get(pgn_piece) # Lấy ký hiệu chuẩn ('P', 'p', None,...)
        if piece_symbol:
            channel_index = piece_to_channel.get(piece_symbol)
            rank, file = square_name_to_index[square_name]
            # Quan trọng: Đảm bảo rank, file đúng với ma trận (hàng, cột)
            # Nếu square_name_to_index map đúng thì rank là chỉ số hàng, file là chỉ số cột
            cnn_array[rank, file, channel_index] = 1.0
    return cnn_array

In [7]:
X = train_df[board_columns] # Chỉ lấy 64 cột bàn cờ
y = train_df[label_column].astype(int)

In [8]:
X_train_df, X_val_df, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [9]:
BATCH_SIZE = 256 # Có thể cần giảm nếu gặp lỗi OOM với CNN
BUFFER_SIZE = 10000

In [10]:
def tf_process_row(board_series_dict, label):
    # board_series_dict là một dict {'a1': tensor, 'b1': tensor,...}
    # Cần chuyển nó về dạng Series hoặc xử lý trực tiếp tensor
    # Cách đơn giản nhất là dùng tf.py_function

    def process_py(*args):
        tensors = args
        # tensors là một list/tuple các tensor string ứng với các cột a1..h8
        # Cần decode và tạo lại cấu trúc Series để đưa vào hàm cũ
        # Lưu ý: tf.py_function có thể làm chậm pipeline, nhưng dễ triển khai
        decoded_strings = [t.numpy().decode('utf-8') for t in tensors]
        board_series_pd = pd.Series(decoded_strings, index=board_columns)
        cnn_arr = board_series_to_cnn_array(board_series_pd)
        return cnn_arr.astype(np.float32)

    # Lấy list các tensor từ dict
    input_tensors = [board_series_dict[col] for col in board_columns]

    cnn_input = tf.py_function(
        func=process_py,
        inp=input_tensors,
        Tout=tf.float32
    )
    # Đảm bảo shape được set đúng sau py_function
    cnn_input.set_shape([8, 8, NUM_CHANNELS])
    return cnn_input, label


In [11]:
def df_to_cnn_dataset(dataframe_X, dataframe_y, shuffle_data=True, batch_size=BATCH_SIZE):
    # Chuyển đổi DataFrame thành dataset các dictionary features
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe_X), dataframe_y))

    if shuffle_data:
        ds = ds.shuffle(buffer_size=BUFFER_SIZE)

    # Áp dụng hàm tiền xử lý để tạo CNN input
    ds = ds.map(tf_process_row, num_parallel_calls=tf.data.AUTOTUNE)

    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE) # Prefetch để tối ưu
    return ds

print("Tạo CNN datasets...")
train_ds = df_to_cnn_dataset(X_train_df, y_train)
val_ds = df_to_cnn_dataset(X_val_df, y_val, shuffle_data=False)
print("Hoàn thành tạo CNN datasets.")

Tạo CNN datasets...
Hoàn thành tạo CNN datasets.


In [12]:
# --- 4. Xây dựng Mô hình CNN ---
def build_cnn_model(input_shape=(8, 8, NUM_CHANNELS)):
    model = tf.keras.Sequential(
        [
            tf.keras.Input(shape=input_shape),
            tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'),
            tf.keras.layers.BatchNormalization(),
            # Có thể thêm lớp Conv2D nữa hoặc MaxPooling2D ở đây
            # layers.MaxPooling2D(pool_size=(2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(256, activation='relu'), # Giữ lại lớp Dense lớn
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(128, activation='relu'), # Lớp Dense thứ 2
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation='sigmoid') # Output layer
        ]
    )
    return model

cnn_model = build_cnn_model()
cnn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 8, 8, 64)       │         6,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 8, 8, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     2,097,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,360,129 (9.00 MB)

 Trainable params: 2,359,489 (9.00 MB)

 Non-trainable params: 640 (2.50 KB)

In [13]:
# --- 5. Biên dịch và Huấn luyện Mô hình CNN ---
cnn_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), # Có thể cần learning rate nhỏ hơn
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("\nBắt đầu huấn luyện mô hình CNN...")

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5, # Tăng patience một chút vì CNN có thể hội tụ chậm hơn
    restore_best_weights=True
)

EPOCHS = 20 # Có thể cần nhiều epochs hơn cho CNN

history = cnn_model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[early_stopping]
)

print("\nHoàn tất huấn luyện CNN.")


Bắt đầu huấn luyện mô hình CNN...
Epoch 1/20
666/666 ━━━━━━━━━━━━━━━━━━━━ 235s 349ms/step - accuracy: 0.9666 - loss: 0.1822 - val_accuracy: 0.9707 - val_loss: 0.1791
Epoch 2/20
666/666 ━━━━━━━━━━━━━━━━━━━━ 230s 345ms/step - accuracy: 0.9710 - loss: 0.1469 - val_accuracy: 0.9707 - val_loss: 0.1582
Epoch 3/20
666/666 ━━━━━━━━━━━━━━━━━━━━ 247s 370ms/step - accuracy: 0.9709 - loss: 0.1430 - val_accuracy: 0.9707 - val_loss: 0.1495
Epoch 4/20
666/666 ━━━━━━━━━━━━━━━━━━━━ 226s 339ms/step - accuracy: 0.9708 - loss: 0.1411 - val_accuracy: 0.9707 - val_loss: 0.1637
Epoch 5/20
666/666 ━━━━━━━━━━━━━━━━━━━━ 264s 396ms/step - accuracy: 0.9711 - loss: 0.1372 - val_accuracy: 0.9707 - val_loss: 0.1495
Epoch 6/20
666/666 ━━━━━━━━━━━━━━━━━━━━ 236s 354ms/step - accuracy: 0.9710 - loss: 0.1378 - val_accuracy: 0.9707 - val_loss: 0.1565
Epoch 7/20
666/666 ━━━━━━━━━━━━━━━━━━━━ 236s 354ms/step - accuracy: 0.9712 - loss: 0.1363 - val_accuracy: 0.9707 - val_loss: 0.1539
Epoch 8/20
666/666 ━━━━━━━━━━━━━━━━━━━━ 2

In [15]:
# --- 6. Đánh giá và Lưu mô hình ---
loss, accuracy = cnn_model.evaluate(val_ds)
print(f"\nCNN Validation Loss: {loss:.4f}")
print(f"CNN Validation Accuracy: {accuracy:.4f}")

# Lưu mô hình CNN
cnn_model_save_path = 'D:/AI/Chess/ChessApp/Chess/model/morphy_cnn' # Đổi tên
os.makedirs(os.path.dirname(cnn_model_save_path), exist_ok=True)
cnn_model.export(cnn_model_save_path) # Lưu định dạng SavedModel (chứa cả kiến trúc và trọng số)
# Hoặc lưu chỉ trọng số: cnn_model.save_weights(cnn_model_save_path + '_weights.h5')
print(f"Đã lưu mô hình CNN vào: {cnn_model_save_path}")

167/167 ━━━━━━━━━━━━━━━━━━━━ 38s 230ms/step - accuracy: 0.9711 - loss: 0.1481

CNN Validation Loss: 0.1495
CNN Validation Accuracy: 0.9707
INFO:tensorflow:Assets written to: D:/AI/Chess/ChessApp/Chess/model/morphy_cnn\assets


INFO:tensorflow:Assets written to: D:/AI/Chess/ChessApp/Chess/model/morphy_cnn\assets


Saved artifact at 'D:/AI/Chess/ChessApp/Chess/model/morphy_cnn'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 8, 8, 12), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2020984635472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2020984637392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2020984637776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2020984637584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2020984636624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2020984636432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2020984638736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2020984639312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2020984639504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2020984637968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2020984636